## Import Module

In [1]:
import numpy as np
import pandas as pd
from vnstock3 import Vnstock
import os

In [2]:
import dea
%load_ext autoreload
%autoreload 2
import pca
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get Top Capalization Companies

In [3]:
def get_top_market_capital_tickers(n_tickers):
    VN_stocks_list = Vnstock().stock().listing.symbols_by_industries()
    non_financial_stocks_list = [stock for stock, code in zip(VN_stocks_list['symbol'], VN_stocks_list['icb_code2']) if not code.startswith('8')]
    
    ratio_data_list = []

    for stock in non_financial_stocks_list:
        try:
            file_path = f"Data/Ratio/{stock}.csv"
            df = pd.read_csv(file_path, header=[0, 1], index_col=0)
            df = df.T.droplevel(0).T
            ratio_data_list.append(df)
        except FileNotFoundError:
            print(f"Không tìm thấy file cho cổ phiếu {stock}")
        except pd.errors.EmptyDataError:
            print(f"File trống cho cổ phiếu {stock}")
        except Exception as e:
            print(f"Lỗi khi tải dữ liệu cho cổ phiếu {stock}: {e}")

    df = pd.concat(ratio_data_list)
    df['quarter']  = df.apply(lambda row: pd.Period(year=row['yearReport'], quarter=row['lengthReport'], freq='Q'), axis=1)
    df.set_index(['quarter', 'ticker'], inplace=True)

    top_market_capital_tickers_dict = {}

    for quarter, group in df.groupby(level='quarter'):
        top_tickers = group.sort_values(by='Market Capital (Bn. VND)', ascending=False).head(n_tickers).index.get_level_values('ticker').tolist()
        top_market_capital_tickers_dict[quarter] = top_tickers

    return top_market_capital_tickers_dict 

## Process Financial Data for DEA Model

In [4]:
def proscess_financial_data(start_quarter, end_quarter, top_tickers_dict, balance_sheet_columns_dict, income_statement_columns_dict):
    year_start, quarter_start = int(start_quarter[:4]), int(start_quarter[-1])
    year_end, quarter_end = int(end_quarter[:4]), int(end_quarter[-1])

    selected_quarters = []
    year, quarter = year_start, quarter_start

    while year < year_end or (year == year_end and quarter <= quarter_end):
        selected_quarters.append(pd.Period(year=year, quarter=quarter, freq='Q'))

        if quarter == 4:
            year += 1
            quarter = 1
        else:
            quarter += 1
    
    data_dict = {}

    for quarter in selected_quarters:
        if quarter not in top_tickers_dict:
            print(f"Quý {quarter} không có dữ liệu trong {top_tickers_dict}.")
            continue
        
        top_tickers = top_tickers_dict[quarter]

        quarter_data = []

        for ticker in top_tickers:
            balance_sheet_file = f"Data/Balance Sheet/{ticker}.csv"
            income_statement_file = f"Data/Income Statement/{ticker}.csv"

            if not os.path.exists(balance_sheet_file) or not os.path.exists(income_statement_file):
                print(f"File cho cổ phiếu {ticker} không tồn tại trong quý {quarter}.")
                continue

            balance_sheet_df = pd.read_csv(balance_sheet_file, header=0, index_col=0)
            if 'yearReport' not in balance_sheet_df.columns or 'lengthReport' not in balance_sheet_df.columns:
                print(f"Thiếu cột 'yearReport' hoặc 'lengthReport' trong dữ liệu của {ticker}.")
                continue

            income_statement_df = pd.read_csv(income_statement_file, header=0, index_col=0)
            if 'yearReport' not in income_statement_df.columns or 'lengthReport' not in income_statement_df.columns:
                print(f"Thiếu cột 'yearReport' hoặc 'lengthReport' trong dữ liệu của {ticker}.")
                continue
            
            balance_sheet_df.loc[:, 'quarter'] = balance_sheet_df.apply(
                lambda row: pd.Period(year=row['yearReport'], quarter=row['lengthReport'], freq='Q'), axis=1)
            income_statement_df.loc[:, 'quarter'] = income_statement_df.apply(
                lambda row: pd.Period(year=row['yearReport'], quarter=row['lengthReport'], freq='Q'), axis=1)

            balance_sheet_filtered = balance_sheet_df[balance_sheet_df['quarter'] == quarter]
            income_statement_filtered = income_statement_df[income_statement_df['quarter'] == quarter]

            if balance_sheet_filtered.empty or income_statement_filtered.empty:
                print('f"Dữ liệu cho cổ phiếu {ticker} không tồn tại trong quý {quarter}')
                continue
            
            try:
                income_statement_filtered['operating expenses'] = income_statement_filtered['General & Admin Expenses'] + income_statement_filtered['Selling Expenses']
            except:
                income_statement_filtered['operating expenses'] = pd.NA
            
            income_statement_filtered['operating expenses'] = pd.to_numeric(income_statement_filtered['operating expenses'], errors='coerce')
            
            valid_balance_sheet_columns = list(balance_sheet_columns_dict.keys())
            
            for col in valid_balance_sheet_columns:
                if col not in balance_sheet_filtered.columns:
                    balance_sheet_filtered[col] = pd.NA

            balance_sheet_filtered = balance_sheet_filtered[['quarter', 'ticker'] + valid_balance_sheet_columns]

            valid_income_statement_columns = list(income_statement_columns_dict.keys())
            
            for col in valid_income_statement_columns:
                if col not in income_statement_filtered.columns:
                    income_statement_filtered[col] = pd.NA

            income_statement_filtered = income_statement_filtered[['quarter', 'ticker', 'operating expenses'] + valid_income_statement_columns]

            merged_data = pd.merge(balance_sheet_filtered, income_statement_filtered, on=['quarter', 'ticker'], how='inner')

            rename_columns = {**balance_sheet_columns_dict, **income_statement_columns_dict}
            merged_data.rename(columns=rename_columns, inplace=True)

            merged_data.loc[:, ['cost of sales', 'operating expenses']] *= -1
            merged_data = merged_data.dropna()
            merged_data = merged_data[merged_data['net income'] > 0]

            quarter_data.append(merged_data)

        if quarter_data:
            data_dict[quarter] = pd.concat(quarter_data, ignore_index=True)
        else:
            print(f"Không có dữ liệu hợp lệ cho quý {quarter}.")

    normalized_data_dict = {}

    for quarter, df in data_dict.items():
        df_nomalized = df.apply(lambda x: x / x.mean() if pd.api.types.is_numeric_dtype(x) else x)
        normalized_data_dict[quarter] = df_nomalized

    return normalized_data_dict
    

## PCA-DEA Model

In [5]:
def calculate_efficiency_scores(data_dict, inputs_variables, outputs_variables, variance_ratio=0.8):
    efficiency_scores_dict = {}

    for quarter in data_dict.keys():
        inputs = data_dict[quarter][inputs_variables]
        outputs = data_dict[quarter][outputs_variables]
        
        tickers = data_dict[quarter]['ticker'].tolist()

        indat_pca = pca.PCA(inputs, variance_ratio=variance_ratio)
        outdat_pca = pca.PCA(outputs, variance_ratio=variance_ratio)
        pcaDEA = dea.DEAProblem(indat_pca, outdat_pca, returns='CRS')
        Results = pcaDEA.solve()
   
        df = pd.DataFrame({'ticker': tickers,
                                    'efficiency_scores': Results['Efficiency']})
    
        efficiency_scores_dict[quarter] = df

    return efficiency_scores_dict

## Applying PCA-DEA for Potential Stocks Selection

In [6]:
top_market_capital_tickers = get_top_market_capital_tickers(100)

2024-10-13 02:32:12,816 - vnstock3.common.vnstock - INFO - Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
2024-10-13 02:32:12,829 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Không tìm thấy file cho cổ phiếu ACS
Không tìm thấy file cho cổ phiếu CKA
Không tìm thấy file cho cổ phiếu AGX
Không tìm thấy file cho cổ phiếu APL
Không tìm thấy file cho cổ phiếu A32
Không tìm thấy file cho cổ phiếu BCV
Không tìm thấy file cho cổ phiếu BBH
Không tìm thấy file cho cổ phiếu BBT
Không tìm thấy file cho cổ phiếu BCP
Không tìm thấy file cho cổ phiếu BLT
Không tìm thấy file cho cổ phiếu BIO
Không tìm thấy file cho cổ phiếu DTB
Không tìm thấy file cho cổ phiếu BMD
Không tìm thấy file cho cổ phiếu BMG
Không tìm thấy file cho cổ phiếu MBN
Không tìm thấy file cho cổ phiếu BTN
Không tìm thấy file cho cổ phiếu BLN
Không tìm thấy file cho cổ phiếu BVN
Không tìm thấy file cho cổ phiếu BWA
Không tìm thấy file cho cổ phiếu C22
Không tìm thấy file cho cổ phiếu HFB
Không tìm thấy file cho cổ phiếu CBS
Không tìm thấy file cho cổ phiếu CDH
Không tìm thấy file cho cổ phiếu TW3
Không tìm thấy file cho cổ phiếu CFM
Không tìm thấy file cho cổ phiếu CH5
Không tìm thấy file cho cổ phiếu CHC
K

In [7]:
start_quarter = '2020Q1'
end_quarter = '2024Q2'

balance_sheet_columns_dict = {'TOTAL ASSETS (Bn. VND)': 'total assets',
                       "OWNER'S EQUITY(Bn.VND)": 'total equity'}

income_statement_columns_dict = {'Cost of Sales': 'cost of sales',
                       'Net Profit For the Year': 'net income',
                       'EPS_basis': 'eps'}

normalized_data_dict = proscess_financial_data(start_quarter, end_quarter, top_market_capital_tickers, balance_sheet_columns_dict, income_statement_columns_dict)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

- Inputs: Total assets, Total equity, Operating expenses, Cost of sales
- Outputs: Net income, EPS

In [8]:
inputs_variables = ['total assets', 'total equity', 'operating expenses', 'cost of sales']
outputs_variables = ['net income', 'eps']

# variance_ratio_dict = {
#     'eff_scores_80': 0.8,
#     'eff_scores_85': 0.85,
#     'eff_scores_90': 0.9,
#     'eff_scores_95': 0.95,
#     'eff_scores_100': 1,
# }

variance_ratio_dict = {
    'eff_scores_70': 0.7,
    'eff_scores_80': 0.8,
    'eff_scores_90': 0.9,
    'eff_scores_100': 1,
}

efficiency_scores_dict = {}

for name, ratio in variance_ratio_dict.items():
    efficiency_scores_dict[name] = calculate_efficiency_scores(normalized_data_dict, inputs_variables, outputs_variables, variance_ratio=ratio)

-----

In [9]:
year = []
quarter = []
n_stocks = []

for key in normalized_data_dict.keys():
    year.append(str(key)[:4])
    quarter.append(str(key)[5:6])
    n_stocks.append(len(normalized_data_dict[key]))

top_cap = pd.DataFrame({
    'Year': year,
    'Quarter': quarter,
    'Number of stocks': n_stocks
})

top_cap.set_index(['Year', 'Quarter'], inplace=True)

top_cap.to_csv('Results/top_cap.csv')

In [10]:
efficiency_stocks_dict = {}
efficiency_desciption_dict = {}

for name, eff_scores in efficiency_scores_dict.items():
    ratio = name.split("_")[-1]

    efficiency_stocks_dict_ = {}

    min = []
    max = []
    mean = []
    n_efficiency = []
    p_efficiency = []

    for key, df in eff_scores.items():
        min.append(df['efficiency_scores'].min())
        max.append(df['efficiency_scores'].max())
        mean.append(df['efficiency_scores'].mean())

        df['efficiency_scores'] = df['efficiency_scores'].apply(lambda x: round(x, 2))
        n_efficiency.append(len(df[df['efficiency_scores'] == 1.00]))
        p_efficiency.append(len(df[df['efficiency_scores'] == 1.00]) / len(df))

        df = df[df['efficiency_scores'] == 1.00]

        year = str(key)[:4]
        quarter = str(key)[5:6]

        if quarter == '4':
            quarter = 1
            year = int(year) + 1
        else:
            quarter = int(quarter) + 1

        efficiency_stocks_dict_[f"{year}_{quarter}"] = df['ticker'].tolist()

    efficiency_desciption = pd.DataFrame({
        'Minimum Efficiency': min,
        'Maximum Efficiency': max,
        'Average Efficiency': mean,
        'Number of Efficient Firms': n_efficiency,
        'Efficient Firms (%)': p_efficiency,
    }, index=top_cap.index)

    new_index = []
    for year, quarter in efficiency_desciption.index:
        if quarter == '4':
            new_index.append((int(year) + 1, 1))
        else:
            new_index.append((int(year), int(quarter) + 1))
    efficiency_desciption.index = pd.MultiIndex.from_tuples(new_index, names=['Year', 'Quarter'])

    efficiency_desciption[['Minimum Efficiency', 'Maximum Efficiency', 'Average Efficiency']] = efficiency_desciption[['Minimum Efficiency', 'Maximum Efficiency', 'Average Efficiency']].map(lambda x: f"{x:.2f}")
    efficiency_desciption['Efficient Firms (%)'] = efficiency_desciption['Efficient Firms (%)'].apply(lambda x: f"{x:.2%}")

    efficiency_desciption_dict[f"efficiency_desciption_{ratio}"] = efficiency_desciption
    efficiency_desciption.to_csv(f'Results/efficiency_description_{ratio}.csv')

    for key in efficiency_stocks_dict_.keys():
        efficiency_stocks_dict_[key] = sorted(efficiency_stocks_dict_[key])

    max_len = np.max(efficiency_desciption['Number of Efficient Firms'])

    for key, lst in efficiency_stocks_dict_.items():
        if len(lst) < max_len:
            lst += [''] * (max_len - len(lst))

    efficiency_stocks = pd.DataFrame(efficiency_stocks_dict_)

    columns = [tuple(col.split('_')) for col in efficiency_stocks.columns]
    efficiency_stocks.columns = pd.MultiIndex.from_tuples(columns)
    efficiency_stocks['No.'] = list(range(1, len(efficiency_stocks) + 1))
    efficiency_stocks.set_index('No.', inplace=True)

    efficiency_stocks_dict[f"efficient_stocks_{ratio}"] = efficiency_stocks
    efficiency_stocks.to_csv(f'Results/efficient_stocks_{ratio}.csv')

In [11]:
efficiency_stocks_dict['efficient_stocks_70']

2020           2021                2022                2023            \
       2    3    4    1    2    3    4    1    2    3    4    1    2    3   
No.                                                                         
1    VCF  VCF  VCF  VCF  HPG  IPA  HPG  VCF  DGC  BSR  ACV  VCF  BMP  GMD   
2                                            HPG  DGC  DGC       BSR  MWG   
3                                            VCF       IDP       GAS  VCF   
4                                                      MSN       PDN  VNM   
5                                                      MWG       REE        
6                                                      VCF       VCF        
7                                                      VNM       VEA        

         2024            
       4    1    2    3  
No.                      
1    ACV  MCH  HVN  ACV  
2    BSR  MWG  VCF  BMP  
3    VCF  VCF       MWG  
4         VNM       VCF  
5                   VNM  
6                        
7

In [12]:
efficiency_desciption_dict['efficiency_desciption_80']

Minimum Efficiency Maximum Efficiency Average Efficiency  \
Year Quarter                                                            
2020 2                     0.24               1.00               0.49   
     3                     0.24               1.00               0.44   
     4                     0.18               1.00               0.38   
2021 1                     0.25               1.00               0.46   
     2                     0.27               1.00               0.57   
     3                     0.17               1.00               0.35   
     4                     0.08               1.00               0.37   
2022 1                     0.31               1.00               0.51   
     2                     0.27               1.00               0.52   
     3                     0.13               1.00               0.46   
     4                     0.16               1.00               0.58   
2023 1                     0.27               1.00               0.58   
     2                     0.19               1.00               0.53   
     3                     0.22               1.00               0.52   
     4                     0.15               1.00               0.45   
2024 1                     0.27               1.00               0.47   
     2                     0.25               1.00               0.49   
     3                     0.28               1.00               0.52   

              Number of Efficient Firms Efficient Firms (%)  
Year Quarter                                                 
2020 2                                4               5.63%  
     3                                3               3.80%  
     4                                3               3.75%  
2021 1                                6               7.06%  
     2                                6               7.23%  
     3                                3               3.61%  
     4                                3               3.70%  
2022 1                                4               4.82%  
     2                                3               3.45%  
     3                                2               2.50%  
     4                                7               9.09%  
2023 1                                5               6.94%  
     2                                8              10.13%  
     3                                4               5.26%  
     4                                3               4.00%  
2024 1                                4               5.19%  
     2                                3               3.85%  
     3                                6               7.32%